In [4]:
import geopandas as gp
import pandas as pd
import uuid
import pickle

raw_data= pd.read_csv("../artifacts/data/raw/raw_bronze_format.csv")
data = gp.GeoDataFrame(raw_data[['geometry', 'ndvi_fn', 'ndvi_value']])
data = data.pivot_table(values=['ndvi_value'], index=['geometry'], columns=['ndvi_fn'], aggfunc='first')
data = data.droplevel(0, axis=1).rename_axis(None, axis=1)
other_cols = raw_data.groupby('geometry')[['crop_name', 'date', 'district', 'tehsil', 'state']].first()
data = data.merge(other_cols, left_index=True, right_index=True, how='inner').reset_index()
feature_order = ['geometry', 'date', 'state', 'district', "tehsil", 'oct_1f', 'oct_2f', 'nov_1f', 'nov_2f', 'dec_1f', 
                 'dec_2f', 'jan_1f', 'jan_2f', 'feb_1f', 'feb_2f', 'mar_1f', 'mar_2f', 'apr_1f', 'apr_2f', 'crop_name']

data = data[feature_order]

data.rename(columns={'crop_name':'crop_type'}, inplace=True)

In [6]:
data.index = data.index.map(lambda x: uuid.uuid4())

with open('../artifacts/data/raw/master_pickle_raw.pkl', 'wb') as file:
    pickle.dump(data.to_dict(), file)

In [8]:
import pickle


with open('../data/master_picke_raw', 'rb') as file:
    raw_data_path = pickle.load(file)

FileNotFoundError: [Errno 2] No such file or directory: '../data/master_picke_raw'

In [9]:
import geopandas as gp

raw_data = gp.GeoDataFrame(raw_data_path)

In [15]:
from copy import deepcopy

def dip_impute(row):
    lst = deepcopy(row)
    act_strt_idx = lst.index.get_loc('oct_2f')
    for i in range(len(lst.loc['oct_2f':'apr_1f'])):
        actual_idx = i + act_strt_idx
        if ((lst.iloc[actual_idx-1] - lst.iloc[actual_idx]) >= 20) and ((lst.iloc[actual_idx+1] - lst.iloc[actual_idx]) >= 20):
            lst.iloc[actual_idx] = (lst.iloc[actual_idx-1] + lst.iloc[actual_idx+1])/2
    return lst

raw_data.iloc[:2,5:-2].apply(dip_impute, axis=1)

,oct_1f,oct_2f,nov_1f,nov_2f,dec_1f,dec_2f,jan_1f,jan_2f,feb_1f,feb_2f,mar_1f,mar_2f,apr_1f
4a761a4a-38cc-4f98-a3a4-1cb7246bc365,189.0,191.0,173.0,126.0,132.0,171.0,183.0,187.0,184.0,180.0,166.0,130.0,120.0
9c842300-c23f-4484-a3aa-a5c8f08f8f06,171.0,166.0,142.0,132.0,165.0,185.0,190.0,188.0,180.0,171.0,143.0,130.0,123.0


In [19]:
row = raw_data.iloc[0,5:-2]

row.iloc[1] = 'gg'

In [20]:
row

oct_1f    189.0
oct_2f       gg
nov_1f    173.0
nov_2f    126.0
dec_1f    132.0
dec_2f    171.0
jan_1f    183.0
jan_2f    187.0
feb_1f    184.0
feb_2f    180.0
mar_1f    166.0
mar_2f    130.0
apr_1f    120.0
Name: 4a761a4a-38cc-4f98-a3a4-1cb7246bc365, dtype: object

In [18]:
row

oct_1f    189.0
oct_2f    191.0
nov_1f    173.0
nov_2f    126.0
dec_1f    132.0
dec_2f    171.0
jan_1f    183.0
jan_2f    187.0
feb_1f    184.0
feb_2f    180.0
mar_1f    166.0
mar_2f    130.0
apr_1f    120.0
Name: 4a761a4a-38cc-4f98-a3a4-1cb7246bc365, dtype: object

In [5]:
kr = {[1,2,3]:4, [4,5,6]:7}

TypeError: unhashable type: 'list'